In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
# Simulation parameters
dt = 0.1; T = 1000

# Initialize arrays
t = np.arange(0, T*dt, dt)
x = np.zeros(T) # Position
v = np.zeros(T) # Velocity
a = np.zeros(T) # Acceleration
e = np.zeros(T) # Disruptive force
u = np.zeros(T) # Control force

In [ ]:
# Simple simulation
np.random.seed(42) # Reproducibility
e = np.random.randn(T) * 0.5

# Control strategy
def simple_control(e_t):
    # For simple control, we apply the inverse of the force just applied
    return -e_t

# Simulation loop
for i in range(1, T):
    # Apply control force
    u[i] = simple_control(e[i])
    
    # Acceleration
    # Force divided by mass, assumed to be unit here
    a[i] = e[i] + u[i]
    
    # Update velocity and posotion
    # v = u + a*t
    v[i] = v[i-1] + a[i] * dt
    # x = u*t + 0.5*a*t**2
    x[i] = x[i-1] + v[i] * dt + 0.5 * a[i] * dt ** 2

# Calculate total error
total_error = np.sum(np.abs(x))

In [ ]:
# Plot the particle's position and velocities
plt.figure(figsize=(12, 8))
plt.subplot(311)
plt.plot(t, x)
plt.title('Particle Position')
plt.ylabel('Position')

plt.subplot(312)
plt.plot(t, e, label='Disruptive Force')
plt.plot(t, u, label='Control Force')
plt.title('Forces')
plt.ylabel('Force')
plt.legend()

plt.subplot(313)
plt.plot(t, np.abs(x))
plt.title('Absolute Error')
plt.xlabel('Time')
plt.ylabel('|Error|')

plt.tight_layout()
plt.show()

print(f"Total error: {total_error}")

In [ ]:
# PID Controller
class PIDController:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.previous_error = 0
        self.integral = 0

    def calculate(self, error):
        self.integral += error * dt
        derivative = (error - self.previous_error) / dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.previous_error = error
        return output

# Simulation function
def simulate(params):
    Kp, Ki, Kd = params
    pid = PIDController(Kp, Ki, Kd)
    
    x = np.zeros(T)
    v = np.zeros(T)
    a = np.zeros(T)
    e = np.random.randn(T) * 0.5  # Random disruptive force
    u = np.zeros(T)

    for i in range(1, T):
        error = -x[i-1]  # Error is negative of position (target is 0)
        u[i] = pid.calculate(error)
        
        a[i] = e[i] + u[i]
        v[i] = v[i-1] + a[i] * dt
        x[i] = x[i-1] + v[i] * dt + 0.5 * a[i] * dt**2

    return x, v, a, e, u

# Objective function for optimization
def objective(params):
    x, _, _, _, _ = simulate(params)
    return np.sum(np.abs(x))

# Optimize PID parameters
initial_guess = [1, 0.1, 0.1]  # Initial guess for Kp, Ki, Kd
result = minimize(objective, initial_guess, method='Nelder-Mead')

optimal_params = result.x
print(f"Optimal PID parameters: Kp={optimal_params[0]:.4f}, Ki={optimal_params[1]:.4f}, Kd={optimal_params[2]:.4f}")
print(f"Minimized total error: {result.fun:.4f}")

# Run simulation with optimized parameters
x, v, a, e, u = simulate(optimal_params)

# Plotting
plt.figure(figsize=(12, 10))
plt.subplot(411)
plt.plot(np.arange(0, T*dt, dt), x)
plt.title('Particle Position')
plt.ylabel('Position')

plt.subplot(412)
plt.plot(np.arange(0, T*dt, dt), v)
plt.title('Particle Velocity')
plt.ylabel('Velocity')

plt.subplot(413)
plt.plot(np.arange(0, T*dt, dt), e, label='Disruptive Force')
plt.plot(np.arange(0, T*dt, dt), u, label='Control Force')
plt.title('Forces')
plt.ylabel('Force')
plt.legend()

plt.subplot(414)
plt.plot(np.arange(0, T*dt, dt), np.abs(x))
plt.title('Absolute Error')
plt.xlabel('Time')
plt.ylabel('|Error|')

plt.tight_layout()
plt.show()

In [ ]:
# Simulation parameters
dt = 0.1  # Time step
T = 1000  # Total number of time steps
delay_steps = int(0.5 / dt)  # Number of time steps for 0.5 second delay

# PID Controller with delay
class DelayedPIDController:
    def __init__(self, Kp, Ki, Kd, delay_steps):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.delay_steps = delay_steps
        self.previous_error = 0
        self.integral = 0
        self.error_history = np.zeros(delay_steps)

    def calculate(self, current_error):
        self.error_history = np.roll(self.error_history, 1)
        self.error_history[0] = current_error
        
        delayed_error = self.error_history[-1]
        self.integral += delayed_error * dt
        derivative = (delayed_error - self.previous_error) / dt
        output = self.Kp * delayed_error + self.Ki * self.integral + self.Kd * derivative
        self.previous_error = delayed_error
        return output

# Simulation function
def simulate(params):
    Kp, Ki, Kd = params
    pid = DelayedPIDController(Kp, Ki, Kd, delay_steps)
    
    x = np.zeros(T)
    v = np.zeros(T)
    a = np.zeros(T)
    e = np.random.randn(T) * 0.5  # Random disruptive force
    u = np.zeros(T)

    for i in range(1, T):
        error = -x[i-1]  # Error is negative of position (target is 0)
        u[i] = pid.calculate(error)
        
        # F = ma, assuming unit mass (m=1), so a = F = e + u
        a[i] = e[i] + u[i]
        v[i] = v[i-1] + a[i] * dt
        x[i] = x[i-1] + v[i] * dt + 0.5 * a[i] * dt**2

    return x, v, a, e, u

# Objective function for optimization
def objective(params):
    x, _, _, _, _ = simulate(params)
    return np.sum(np.abs(x))

# Optimize PID parameters
initial_guess = [1, 0.1, 0.1]  # Initial guess for Kp, Ki, Kd
result = minimize(objective, initial_guess, method='Nelder-Mead')

optimal_params = result.x
print(f"Optimal PID parameters: Kp={optimal_params[0]:.4f}, Ki={optimal_params[1]:.4f}, Kd={optimal_params[2]:.4f}")
print(f"Minimized total error: {result.fun:.4f}")

# Run simulation with optimized parameters
x, v, a, e, u = simulate(optimal_params)

# Plotting
plt.figure(figsize=(12, 15))
plt.subplot(511)
plt.plot(np.arange(0, T*dt, dt), x)
plt.title('Particle Position')
plt.ylabel('Position')

plt.subplot(512)
plt.plot(np.arange(0, T*dt, dt), v)
plt.title('Particle Velocity')
plt.ylabel('Velocity')

plt.subplot(513)
plt.plot(np.arange(0, T*dt, dt), a)
plt.title('Particle Acceleration')
plt.ylabel('Acceleration')

plt.subplot(514)
plt.plot(np.arange(0, T*dt, dt), e, label='Disruptive Force', alpha=0.7)
plt.plot(np.arange(0, T*dt, dt), u, label='Control Force', alpha=0.7)
plt.title('Forces')
plt.ylabel('Force')
plt.legend()

plt.subplot(515)
plt.plot(np.arange(0, T*dt, dt), np.abs(x))
plt.title('Absolute Error')
plt.xlabel('Time')
plt.ylabel('|Error|')

plt.tight_layout()
plt.show()

In [ ]:
plt.plot(np.arange(0, T*dt, dt), e, label='Disruptive Force', alpha=0.7)
plt.plot(np.arange(0, T*dt, dt), u, label='Control Force', alpha=0.7)
plt.title('Forces')
plt.ylabel('Force')
plt.legend()

In [ ]:
plt.plot(e)

In [ ]:
max(u)